In [ ]:
%load_ext autoreload
%autoreload 2

# White background for matplotlib plots
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

import numpy as np
import tensorflow as tf
import torch

import thesis.quantization_benchmark as qb

In [ ]:
dense_model = tf.keras.Sequential(
    [
        tf.keras.layers.Input((32, 32, 3)),
        # tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu),
        # tf.keras.layers.GlobalMaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dense(128),
    ]
)

# dense_model_torch = torch.nn.Sequential(
#     torch.nn.Flatten(),
#     tf.keras.layers.Dense(256, activation=tf.nn.relu),
#     torch.nn.Linear(28 * 28, 512),
#     torch.nn.ReLU(),
#     torch.nn.Linear(512, 512),
#     torch.nn.ReLU(),
#     torch.nn.Linear(512, 10),
# )

runtimes = [
    qb.TensorFlow(),
    qb.TFLite(quantization_mode="off"),
    qb.TFLite(quantization_mode="dynamic"),
    qb.TFLite(quantization_mode="static"),
    qb.ONNXRuntime(quantization_mode="off"),
    qb.ONNXRuntime(quantization_mode="dynamic"),
    qb.ONNXRuntime(quantization_mode="static_qoperator"),
    qb.ONNXRuntime(quantization_mode="static_qdq"),
]

times = qb.benchmark(dense_model, runtimes)

In [ ]:
import matplotlib.pyplot as plt
for t in times:
    plt.plot(t.times)
    plt.show()

## CNN model

In [ ]:
cnn_model = tf.keras.Sequential(
    [
        tf.keras.layers.Input((32, 32, 3)),
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu),
        tf.keras.layers.GlobalMaxPooling2D(),
        # tf.keras.layers.Dense(128),
    ]
)

runtimes = [
    qb.TensorFlow(),
    qb.TFLite(quantization_mode="off"),
    qb.TFLite(quantization_mode="dynamic"),
    qb.TFLite(quantization_mode="static"),
    qb.ONNXRuntime(quantization_mode="off"),
    # qb.ONNXRuntime(quantization_mode="dynamic"),
    qb.ONNXRuntime(quantization_mode="static_qoperator"),
    qb.ONNXRuntime(quantization_mode="static_qdq"),
]

qb.benchmark(cnn_model, runtimes)

In [ ]:
data = np.random.randn(*([1] + list(model.input.shape)[1:]))
runtimes[0].run(data)